Kawasaki Quantum Summer Camp 2025

# 量子化学シミュレーション：サンプルベースの量子対角化

Kifumi Numata, IBM Quantum (Jul 31, 2025)

In [ ]:
# Import common packages first
import numpy as np
from math import comb
import warnings
import pyscf
import matplotlib.pyplot as plt
import pickle
from functools import partial

# Import qiskit classes
from qiskit import QuantumCircuit, QuantumRegister
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.visualization import plot_gate_map
from qiskit_addon_sqd.fermion import SCIResult, diagonalize_fermionic_hamiltonian, solve_sci_batch

# Import qiskit ecosystems
import ffsim
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler
from qiskit_ibm_runtime import SamplerOptions

In [ ]:
import qiskit
print(f"Qiskit version: {qiskit.__version__}")

**目次**

- 1: チュートリアル -  $N_2$ 分子の基底エネルギーを近似する手法を学ぶ
- 2: 演習 - パラメーターを調整して、基底エネルギーの改善をする

# 1: チュートリアル

## Step 1. 問題を量子回路にマッピングする

まず、$N_2$ 分子とそのプロパティを指定します。次に、量子回路を作成し、量子コンピューターから基底状態エネルギー推定用のサンプルを生成します。

<div class="alert alert-block alert-info">
    
⚠️ **Note:** 回路データは事前に計算されてこのノートブックで提供されているため、`pyscf` の実行の必要はありません。下のセルを実行して、**「最適化された量子回路をロードする」** セクションに直接進むことができます。
</div>


In [ ]:
import warnings, pickle
import numpy as np
warnings.filterwarnings("ignore")

# from pyscf import ao2mo, tools

# 分子特性の指定
num_orbitals = 16
num_elec_a = num_elec_b = 5
open_shell = False
spin_sq = 0
nelec = (num_elec_a, num_elec_b)

# pyscfを使用する場合は、ディスクから分子を読み込む
# mf_as = tools.fcidump.to_scf("utils/n2_fci.txt")
# hcore = mf_as.get_hcore()
# eri = ao2mo.restore(1, mf_as._eri, num_orbitals)
# nuclear_repulsion_energy = mf_as.mol.energy_nuc()

# pyscfを使用しない場合は、値をロードする
hcore = np.load("utils/hcore.npy") # 1電子ハミルトニアン
eri = np.load("utils/eri.npy") # 2電子ハミルトニアン
with open("utils/nuclear_repulsion_energy.pkl", "rb") as f:
    nuclear_repulsion_energy = pickle.load(f) # 原子核エネルギー

## Step 2. 量子回路をロードする

このチュートリアルでは、事前に準備された量子回路から始めます。 量子回路は、$N_2$ 分子のハミルトニアンをもとに近似的に作成されています。またこのアンサッツは、IBM のヘビーヘックストポロジーを念頭に置いて作成されており、量子ビットの相互作用に **ジグザグパターン** を採用しています。 このパターンでは、同じ電子スピンの軌道がヘビーヘックスの格子全体にジグザグ線で接続されています (下図参照)。

![lucj_ansatz](https://quantum.cloud.ibm.com/assets-docs-learning/_next/image?url=%2Fdocs%2Fimages%2Ftutorials%2Fimproving-energy-estimation-of-a-fermionic-hamiltonian-with-sqd%2F7e0ee7e1-2d24-417f-ac59-25c58db79aa9.avif&w=1920&q=75)

In [ ]:
from qiskit import qpy
 
with open('utils/N2_UCJ_Transpiled.qpy', 'rb') as fd:
    isa_circuit = qpy.load(fd)[0]

## Step 3. 実験を実行する

ハードウェア実行用に回路を最適化したら、ターゲット ハードウェア上で回路を実行し、基底状態のエネルギー推定のためのサンプルを収集する準備が整います。

<div class="alert alert-block alert-warning">
    
⚠️ **Note:** 量子コンピューター上で回路を実行するためのコードはコメントアウトして、ユーザーの参照用に残してあります。このチュートリアルでは、実デバイス上で実行するのではなく、以前に ``ibm_sherbrooke`` から取得した 10 万個のサンプルを読み込みます。

</div>


In [ ]:
import numpy as np

# from qiskit_ibm_runtime import SamplerV2 as Sampler

# sampler = Sampler(mode=backend)
# job = sampler.run([isa_circuit], shots=100_000)
# primitive_result = job.result()
# pub_result = primitive_result[0]
# counts = pub_result.data.meas.get_counts()

bit_array = np.load('utils/N2_device_bitarray.npy', allow_pickle=True).item()

## Step 4. サンプルベースの量子対角化 (SQD) を使用した結果の後処理

このセクションでは、量子回路から生成・測定された結果を後処理して、分子の基底状態エネルギーを推定します。データサンプルを修正することを繰り返し、精度を向上させます。

ここで、ユーザーが設定できるオプションがいくつかあります。

* `max_iterations`: ループの反復回数。
* `num_batches`: 取り出すバッチの数 
* `samples_per_batch`: 各バッチに含めるサンプル数
* `max_cycles`: 固有状態ソルバーの最大サイクル数


In [ ]:
%%time
# SQDのオプション
energy_tol = 1e-3  
occupancies_tol = 1e-3 
max_iterations = 5

# 固有状態ソルバーのオプション
num_batches = 5
samples_per_batch = 50
symmetrize_spin = True 
carryover_threshold = 1e-4 
max_cycle = 200
rng = np.random.default_rng(24)

# 中間結果を保存するためのリスト
result_history = [] 

def callback(results: list[SCIResult]): 
    result_history.append(results)
    iteration = len(result_history)
    print(f"Iteration {iteration}")
    for i, result in enumerate(results):
        print(f"\tSubsample {i}")
        print(f"\t\tEnergy: {result.energy + nuclear_repulsion_energy}")
        print(f"\t\tSubspace dimension: {np.prod(result.sci_state.amplitudes.shape)}")

result = diagonalize_fermionic_hamiltonian(
    hcore,
    eri,
    bit_array,
    samples_per_batch=samples_per_batch,
    norb=num_orbitals,
    nelec=nelec,
    num_batches=num_batches,
    energy_tol=energy_tol,
    occupancies_tol=occupancies_tol,
    max_iterations=max_iterations,
    symmetrize_spin=symmetrize_spin,
    carryover_threshold=carryover_threshold,
    callback=callback,
    seed=rng,
)

### 結果を表示する

In [ ]:
def plot_energy_and_occupancy(result_history, exact_energy):

    # Data for energies plot
    
    x1 = range(len(result_history))
    min_e = [
        min(result, key=lambda res: res.energy).energy + nuclear_repulsion_energy
        for result in result_history
    ]
    e_diff = [abs(e - exact_energy) for e in min_e]
    yt1 = [1.0, 1e-1, 1e-2, 1e-3, 1e-4]
    
    # Chemical accuracy (+/- 1 milli-Hartree)
    chem_accuracy = 0.001
    
    # Data for avg spatial orbital occupancy
    y2 = np.sum(result.orbital_occupancies, axis=0)
    x2 = range(len(y2))
    
    fig, axs = plt.subplots(1, 2, figsize=(12, 6))
    
    # Plot energies
    axs[0].plot(x1, e_diff, label="energy error", marker="o")
    axs[0].set_xticks(x1)
    axs[0].set_xticklabels(x1)
    axs[0].set_yticks(yt1)
    axs[0].set_yticklabels(yt1)
    axs[0].set_yscale("log")
    axs[0].set_ylim(1e-4)
    axs[0].axhline(y=chem_accuracy, color="#BF5700", linestyle="--", label="chemical accuracy")
    axs[0].set_title("Approximated Ground State Energy Error vs SQD Iterations")
    axs[0].set_xlabel("Iteration Index", fontdict={"fontsize": 12})
    axs[0].set_ylabel("Energy Error (Ha)", fontdict={"fontsize": 12})
    axs[0].legend()
    
    # Plot orbital occupancy
    axs[1].bar(x2, y2, width=0.8)
    axs[1].set_xticks(x2)
    axs[1].set_xticklabels(x2)
    axs[1].set_title("Avg Occupancy per Spatial Orbital")
    axs[1].set_xlabel("Orbital Index", fontdict={"fontsize": 12})
    axs[1].set_ylabel("Avg Occupancy", fontdict={"fontsize": 12})
    
    print(f"Exact energy: {exact_energy:.5f} Ha")
    print(f"SQD energy: {min_e[-1]:.5f} Ha")
    print(f"Absolute error: {e_diff[-1]:.5f} Ha")
    plt.tight_layout()
    plt.show()

In [ ]:
exact_energy = -109.10288938
plot_energy_and_occupancy(result_history, exact_energy)

最初のプロットは、得られたエネルギーの近似値が化学的精度からどの程度離れているかを示しています。（一般的には ``1 kcal/mol`` $\approx$ ``1.6 mH``).

2 番目のプロットは、最終の反復後の各空間軌道の平均占有率を示しています。私たちの解では、電子が二つずつ、最初の 5 つの軌道を高い確率で占有していることがわかります。

# 2. 演習 

エネルギー推定の精度を向上させるという課題に取り組みます。Part 1  の初期設定で使ったいくつかのパラメーターを変化させることでパフォーマンスにどのように影響するかを探ります。

<a id="exercise_1"></a>
<div class="alert alert-block alert-success">
    
<b>基底状態エネルギー推定の改良</b> 

チュートリアルの例では、小さなハミルトニアンを使用し、1回の反復で、バッチ数を 5 にして、分子の基底状態エネルギーの初期近似値を取得しました。ただし、精度を向上させる余地は大きくあります。**Part 2**のタスクは、次の2つの主要なパラメーターを試して、エネルギー推定値を向上させることです。

- **サンプリングサイズ** (`samples_per_batch`): バッチあたりのサンプル数を調整して、ハミルトニアンのサイズを定義します。この値を大きくすると精度が向上しますが、計算負荷が増加する可能性があります。
- **バッチ数** (`num_batches`): バッチ数を変更して、各反復での計算効率と結果の精度のバランスを見つけます。

これらのパラメーターを戦略的に使用して、推定される基底状態エネルギーを実際の値にできるだけ近づけることを目指します。ノイズやハードウェア制約の影響を考慮しながら、さまざまな組み合わせをテストしてエラーを最小限に抑えます。

真の基底状態エネルギーにどれだけ近づけるか見てみましょう。
</div>

まず、上記と同じループを実行し、``num_batches`` および ``samples_per_batch`` 引数を入力として受け取る関数を定義します。

In [ ]:
def sqd_configuration_recovery(num_batches: int, samples_per_batch: int) -> float:
    # SQDのオプション
    energy_tol = 1e-3  
    occupancies_tol = 1e-3 
    max_iterations = 5
    
    # 固有状態ソルバーのオプション
    symmetrize_spin = True 
    carryover_threshold = 1e-4 
    max_cycle = 200
    rng = np.random.default_rng(24)
    
    result = diagonalize_fermionic_hamiltonian(
        hcore,
        eri,
        bit_array,
        samples_per_batch=samples_per_batch,
        norb=num_orbitals,
        nelec=nelec,
        num_batches=num_batches,
        energy_tol=energy_tol,
        occupancies_tol=occupancies_tol,
        max_iterations=max_iterations,
        symmetrize_spin=symmetrize_spin,
        carryover_threshold=carryover_threshold,
        callback=callback,
        seed=rng,
    )
    return result_history

In [ ]:
### この下にコードを記入してください ###

num_batches = 5
samples_per_batch = 100

### この行以降のコードは変更しないでください ###

In [ ]:
%%time
result_history = [] 
energy_hist = sqd_configuration_recovery(num_batches, samples_per_batch)

In [ ]:
plot_energy_and_occupancy(result_history, exact_energy)

1章のチュートリアルの結果は以下でした。

    Exact energy: -109.10289 Ha
    SQD energy: -109.02333 Ha
    Absolute error: 0.07956 Ha

あなたの演習の結果は、より良くなりましたか？

# オプション：実機での実行

分子とその特性を特定します。

In [ ]:
warnings.filterwarnings("ignore")

# 分子特性を指定する
open_shell = False
spin_sq = 0

# N2分子を構築する
mol = pyscf.gto.Mole()
mol.build(
    atom=[["N", (0, 0, 0)], ["N", (1.0, 0, 0)]],
    basis="6-31g",
    symmetry="Dooh",
)

# アクティブスペースを定義する
n_frozen = 2
active_space = range(n_frozen, mol.nao_nr())

# 分子積分を計算する
scf = pyscf.scf.RHF(mol).run()
num_orbitals = len(active_space)
n_electrons = int(sum(scf.mo_occ[active_space]))
num_elec_a = (n_electrons + mol.spin) // 2
num_elec_b = (n_electrons - mol.spin) // 2
cas = pyscf.mcscf.CASCI(scf, num_orbitals, (num_elec_a, num_elec_b))
mo = cas.sort_mo(active_space, base=0)
hcore, nuclear_repulsion_energy = cas.get_h1cas(mo)
eri = pyscf.ao2mo.restore(1, cas.get_h2cas(mo), num_orbitals)

# 厳密解を計算する
exact_energy = cas.run().e_tot

量子回路を構築する際のパラメーターを計算します。

In [ ]:
# 回路を初期化するためのCCSD t1, t2振幅を得る
ccsd = pyscf.cc.CCSD(scf, frozen=[i for i in range(mol.nao_nr()) if i not in active_space]).run()
t1 = ccsd.t1
t2 = ccsd.t2

[ffsim](https://github.com/qiskit-community/ffsim)を使用して量子回路を作成します。

In [ ]:
n_reps = 1
alpha_alpha_indices = [(p, p + 1) for p in range(num_orbitals - 1)]
alpha_beta_indices = [(p, p) for p in range(0, num_orbitals, 4)]

ucj_op = ffsim.UCJOpSpinBalanced.from_t_amplitudes(
    t2=t2,
    t1=t1,
    n_reps=n_reps,
    interaction_pairs=(alpha_alpha_indices, alpha_beta_indices),
)

nelec = (num_elec_a, num_elec_b)

# create an empty quantum circuit
qubits = QuantumRegister(2 * num_orbitals, name="q")
circuit = QuantumCircuit(qubits)

# prepare Hartree-Fock state as the reference state and append it to the quantum circuit
circuit.append(ffsim.qiskit.PrepareHartreeFockJW(num_orbitals, nelec), qubits)

# apply the UCJ operator to the reference state
circuit.append(ffsim.qiskit.UCJOpSpinBalancedJW(ucj_op), qubits)
circuit.measure_all()

circuit.decompose().decompose().draw("mpl", fold =-1)

量子コンピューターのデバイスを指定します。ここでは、127量子ビットのデバイス、「ibm_brisbane」か「ibm_sherbrooke」を使います。

In [ ]:
service = QiskitRuntimeService()
service.backends()

In [ ]:
# 以下でデバイスを指定します。
backend = service.backend('ibm_brisbane') 
#backend = service.backend('ibm_sherbrooke') 

デバイス上の物理量子ビットを指定し、回路を実機で実行できるようにトランスパイルします。

In [ ]:
spin_a_layout = [0, 14, 18, 19, 20, 33, 39, 40, 41, 53, 60, 61, 62, 72, 81, 82]
spin_b_layout = [2, 3, 4, 15, 22, 23, 24, 34, 43, 44, 45, 54, 64, 65, 66, 73]
initial_layout = spin_a_layout + spin_b_layout

pass_manager = generate_preset_pass_manager(
    optimization_level=3, backend=backend, initial_layout=initial_layout
)

# このパスマネージャーによって生成された回路をハードウェア実行に使用します。
pass_manager.pre_init = ffsim.qiskit.PRE_INIT
isa_circuit = pass_manager.run(circuit)
print(f"Gate counts (w/ pre-init passes): {isa_circuit.count_ops()}")

回路を実行します。

In [ ]:
opts = SamplerOptions()
opts.dynamical_decoupling.enable = True
opts.twirling.enable_measure = True

sampler = Sampler(mode=backend, options=opts)
job = sampler.run([isa_circuit], shots=100_000)
print("job id:", job.job_id())
job.status()

In [ ]:
job = service.job(job.job_id())
job.status() # ジョブの実行状態を確認します

上記のセルを何回か実行して、'DONE' が表示されたら、実機での実行が終わっているので、以下のセルを実行します。

In [ ]:
### 'DONE'になってから実行します ###
%%time
result_history = [] 
energy_hist = sqd_configuration_recovery(num_batches, samples_per_batch)

In [ ]:
plot_energy_and_occupancy(result_history, exact_energy)